In [74]:
!pip install pandas
!pip install nltk
!pip install fasttext-wheel

In [122]:
#import dependencies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import re
import string
from nltk.tokenize import RegexpTokenizer # regexp tokenizers use to split words from text
from nltk.stem.snowball import SnowballStemmer # stemmes words
import tensorflow as tf

In [76]:
import tensorflow as tf

In [77]:
print(tf.config.list_physical_devices())

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [78]:
print(tf.test.is_built_with_cuda())

True


In [79]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Restrict TensorFlow to only allocate memory as needed
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU(s) is/are being used.")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
else:
    print("No GPU found. Using CPU.")

GPU(s) is/are being used.


In [80]:
import gc

In [81]:
import tensorflow as tf

tf.keras.backend.clear_session()

In [82]:
import gc

gc.collect()

3380

In [83]:
import fasttext

In [84]:
data_fake = pd.read_csv('Fake.csv')
data_true = pd.read_csv('True.csv')

In [85]:
data_fake["class"] = 0
data_true["class"] = 1

In [86]:
data_fake.shape, data_true.shape

((23481, 5), (21417, 5))

In [87]:
data_merge = pd.concat([data_fake, data_true], axis = 0)
data_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [88]:
data = data_merge.drop(['title', 'subject', 'date'], axis = 1)

In [89]:
data.isnull().sum()

text     0
class    0
dtype: int64

In [90]:
data = data.sample(frac = 1)

data.reset_index(inplace = True)
data.drop(['index'], axis = 1, inplace = True)

In [91]:
data.head()

,text,class
0,Campaign volunteers are key to a winning presi...,0
1,(This January 3 story was corrected to remove...,1
2,Conservatives just love to be stupid. They can...,0
3,WASHINGTON (Reuters) - Republican Senator Susa...,1
4,President Obama has finally been vindicated af...,0


In [92]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]','', text) #remove text enclosed in square brackets, including the brackets themselves
    text = re.sub("\\W", " ", text) #replaces all non-word characters (e.g., punctuation, special characters, symbols) with spaces
    text = re.sub('https?://\S+|www\.\S+', '', text) # removes URLs from the text by matching and removing both HTTP/HTTPS URLs and "www" URLs
    text = re.sub('<.*?>+', '', text) #remove HTML tags and their contents
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) #remove all punctuation characters
    text = re.sub('\n', '', text) #removes newline characters, which are typically used to represent line breaks or paragraphs in text
    text = re.sub('\w*\d\w', '', text) #removes words containing numbers or alphanumeric patterns
    return text

In [93]:
data['text'] = data['text'].apply(wordopt)

In [94]:
tokenizer = RegexpTokenizer(r'[A-Za-z]+')#to getting alpha only
stemmer = SnowballStemmer("english")
data['text'] = data.text.map(lambda t: tokenizer.tokenize(t))
data['text'] = data.text.map(lambda l: [stemmer.stem(word) for word in l])
data['text'] = data.text.map(lambda l: ' '.join(l))

In [95]:
data['class_label'] = "__label__" + data['class'].astype(str)
data.head()

,text,class,class_label
0,campaign volunt are key to a win presidenti ca...,0,__label__0
1,this januari stori was correct to remov refer ...,1,__label__1
2,conserv just love to be stupid they can t help...,0,__label__0
3,washington reuter republican senat susan colli...,1,__label__1
4,presid obama has final been vindic after seven...,0,__label__0


In [96]:
data['class_text'] = data['class_label'] + " " + data['text']
data.head()

,text,class,class_label,class_text
0,campaign volunt are key to a win presidenti ca...,0,__label__0,__label__0 campaign volunt are key to a win pr...
1,this januari stori was correct to remov refer ...,1,__label__1,__label__1 this januari stori was correct to r...
2,conserv just love to be stupid they can t help...,0,__label__0,__label__0 conserv just love to be stupid they...
3,washington reuter republican senat susan colli...,1,__label__1,__label__1 washington reuter republican senat ...
4,presid obama has final been vindic after seven...,0,__label__0,__label__0 presid obama has final been vindic ...


In [97]:
random_seed = 42

In [98]:
train, test = train_test_split(data, test_size = 0.25, random_state=random_seed)

In [99]:
x_train, x_test, y_train, y_test = train_test_split(data['text'], data['class'], test_size=0.25, random_state=random_seed)

In [100]:
train.to_csv("news.train", columns=["class_text"], index=False, header=False)
test.to_csv("news.test", columns=["class_text"], index=False, header=False)

In [101]:
# model = fasttext.train_supervised(input="news.train")
# model.test("news.test")
model = fasttext.train_supervised(input="news.train")
model.test("news.test")

(11225, 0.9910913140311804, 0.9910913140311804)

In [102]:
model.save_model("news_model.bin")

In [103]:
model = fasttext.load_model('news_model.bin')

In [104]:
model.predict('Donald Trump spent a good portion of his day at his golf club, marking the 84th day he s done so since taking the oath of office. It must have been a bad game because just after that, Trump lashed out at FBI Deputy Director Andrew McCabe on Twitter following a report saying McCabe plans to retire in a few months. The report follows McCabe s testimony in front of congressional committees this week, as well as mounting criticism from Republicans regarding the Russia probe.So, naturally, Trump attacked McCabe with a lie. How can FBI Deputy Director Andrew McCabe, the man in charge, along with leakin  James Comey, of the Phony Hillary Clinton investigation (including her 33,000 illegally deleted emails) be given $700,000 for wife s campaign by Clinton Puppets during investigation?  Trump tweeted.How can FBI Deputy Director Andrew McCabe, the man in charge, along with leakin  James Comey, of the Phony Hillary Clinton investigation (including her 33,000 illegally deleted emails) be given $700,000 for wife s campaign by Clinton Puppets during investigation?  Donald J. Trump (@realDonaldTrump) December 23, 2017He didn t stop there.FBI Deputy Director Andrew McCabe is racing the clock to retire with full benefits. 90 days to go?!!!  Donald J. Trump (@realDonaldTrump) December 23, 2017Wow,  FBI lawyer James Baker reassigned,  according to @FoxNews.  Donald J. Trump (@realDonaldTrump) December 23, 2017With all of the Intel at Trump s disposal, he s getting his information from Fox News. McCabe spent most of his career in the fight against terrorism and now he s being attacked by the so-called president. Trump has been fact-checked before on his claim of his wife receiving $700,000 for her campaign.Politifact noted in late July that Trump s  tweet about Andrew McCabe is a significant distortion of the facts. And the implication that McCabe got Clinton off as a political favor doesn t make much sense when we look at the evidence. His July tweet was rated  mostly false.  But Trump repeats these lies because he knows his supporters will believe them without bothering to Google. It s still a lie, though.Photo by Zach Gibson   Pool/Getty Images.')

(('__label__0',), array([0.99987304]))

In [105]:
# Extract vocabulary and embeddings
vocabulary = model.get_words()
word_embeddings = np.array([model.get_word_vector(word) for word in vocabulary])

In [106]:
# Create the embedding matrix
embedding_dim = model.get_dimension()  # Dimension of FastText embeddings
embedding_matrix = np.zeros((len(vocabulary), embedding_dim))
for i, word in enumerate(vocabulary):
    embedding_matrix[i] = model.get_word_vector(word)

In [107]:
# Save the embedding matrix to a file
np.save('embedding_matrix.npy', embedding_matrix)  #file is in the NumPy array format as .npy

In [108]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.models import Sequential
from keras.layers import Dropout, Flatten


In [109]:
text_column = data['text']

# Concatenate all the text data into a single string
all_text = ' '.join(text_column)

# Split the text into words and count them
word_count = len(all_text.split())
print(word_count)

18406124


In [110]:
max_sequence_length = 0
for text in text_column:
    tokens = tokenizer.tokenize(text)
    sequence_length = len(tokens)
    max_sequence_length = max(max_sequence_length, sequence_length)

In [111]:
print(max_sequence_length)

8280


In [112]:
vocab_size = len(vocabulary)
sequence_length = max_sequence_length

In [113]:
embedding_matrix.shape

(75033, 100)

In [114]:
print(type(x_train))

<class 'pandas.core.series.Series'>


In [115]:
# Convert Pandas Series to NumPy arrays
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

In [116]:
# Check the data types and shapes to verify they are suitable
print("x_train type:", type(x_train))
print("x_train shape:", x_train.shape)
print("y_train type:", type(y_train))
print("y_train shape:", y_train.shape)
print("x_test type:", type(x_test))
print("x_test shape:", x_test.shape)
print("y_test type:", type(y_test))
print("y_test shape:", y_test.shape)

x_train type: <class 'numpy.ndarray'>
x_train shape: (33673,)
y_train type: <class 'numpy.ndarray'>
y_train shape: (33673,)
x_test type: <class 'numpy.ndarray'>
x_test shape: (11225,)
y_test type: <class 'numpy.ndarray'>
y_test shape: (11225,)


In [117]:
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [118]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)  # Assuming x_train is a list of text strings
x_train_sequences = tokenizer.texts_to_sequences(x_train)
x_test_sequences = tokenizer.texts_to_sequences(x_test)

# Pad sequences
x_train_padded = pad_sequences(x_train_sequences, maxlen=sequence_length)
x_test_padded = pad_sequences(x_test_sequences, maxlen=sequence_length)

In [130]:
import pickle

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [119]:
from keras.models import Model
from keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Concatenate, GlobalMaxPooling1D, Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [120]:
# Input layer
input_layer = Input(shape=(sequence_length,))

# Embedding layer
embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], trainable=False)(input_layer)

# Convolutional layers
conv1d = Conv1D(filters=128, kernel_size=3, activation='relu')(embedding)
conv1d_1 = Conv1D(filters=128, kernel_size=4, activation='relu')(embedding)
conv1d_2 = Conv1D(filters=128, kernel_size=5, activation='relu')(embedding)

# MaxPooling layers
max_pooling1d = MaxPooling1D(pool_size=2)(conv1d)
max_pooling1d_1 = MaxPooling1D(pool_size=2)(conv1d_1)
max_pooling1d_2 = MaxPooling1D(pool_size=2)(conv1d_2)

# Concatenate max-pooled layers
concatenated = Concatenate(axis=1)([max_pooling1d, max_pooling1d_1, max_pooling1d_2])

# Additional Convolutional and MaxPooling layers
conv1d_3 = Conv1D(filters=128, kernel_size=3, activation='relu')(concatenated)
max_pooling1d_3 = MaxPooling1D(pool_size=2)(conv1d_3)
conv1d_4 = Conv1D(filters=128, kernel_size=3, activation='relu')(max_pooling1d_3)
max_pooling1d_4 = MaxPooling1D(pool_size=2)(conv1d_4)

# GlobalMaxPooling layer
global_max_pooling1d = GlobalMaxPooling1D()(max_pooling1d_4)

# Dense layers
dense = Dense(128, activation='relu')(global_max_pooling1d)
output_layer = Dense(1, activation='sigmoid')(dense)  # Assuming binary classification

# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model (specify loss, optimizer, and metrics)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Define model checkpoint
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

# Print a summary of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 8280)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, 8280, 100)    7503300     ['input_1[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 8278, 128)    38528       ['embedding[0][0]']              
                                                                                                  
 conv1d_1 (Conv1D)              (None, 8277, 128)    51328       ['embedding[0][0]']              
                                                                                              

In [ ]:
model.fit(x_train_padded, y_train, validation_data=(x_test_padded, y_test), epochs=5, batch_size=20, callbacks=[early_stopping, model_checkpoint])

In [126]:
from tensorflow.keras.models import load_model
model1 = load_model(r"C:\Users\Atharva\Downloads\MLprac\ML\project\isot fake news\models\fasttext-cnn-model.h5")


In [127]:
loss, accuracy = model1.evaluate(x_test_padded, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

351/351 [==============================] - 24s 68ms/step - loss: 0.0163 - accuracy: 0.9945
Test Loss: 0.016325317323207855, Test Accuracy: 0.9944766163825989


In [54]:
model.save('models/fasttext-cnn-model.h5')

In [128]:
tokenizer = RegexpTokenizer(r'[A-Za-z]+') #to getting alpha only
stemmer = SnowballStemmer("english")

# Tokenize and preprocess the new input data
new_input_data = [ "WASHINGTON (Reuters) - A Georgian-American businessman who met then-Miss Universe pageant owner Donald Trump in 2013, has been questioned by congressional investigators about whether he helped organize a meeting between Russians and Trumpâ€™s eldest son during the 2016 election campaign, four sources familiar with the matter said. The meeting at Trump Tower in New York involving Donald Trump Jr. and other campaign advisers is a focus of probes by Congress and Special Counsel Robert Mueller on whether campaign officials colluded with Russia when it sought to interfere in the U.S. election, the sources said. Russia denies allegations by U.S. intelligence agencies that it meddled in the election and President Donald Trump denies any collusion. The Senate and House of Representatives intelligence committees recently questioned behind closed doors Irakly Kaveladze, a U.S. citizen born in the former Soviet republic of Georgia, the sources said. He is a U.S.-based representative of Azerbaijani oligarch Aras Agalarovâ€™s real estate firm, the Crocus Group. The panels knew Kaveladze was at the June 9, 2016 meeting but became more interested in him after learning he also attended a private dinner in Las Vegas in 2013 with Trump and Agalarov as they celebrated an agreement to hold that yearâ€™s Miss Universe pageant in Moscow, the sources said.  Committee members now want to know more about the extent of Kaveladzeâ€™s contacts with the Trump family and whether he had a bigger role than previously believed in setting up the Trump Tower meeting when Trump was a Republican candidate for president. The White House declined to comment. Muellerâ€™s office also declined to comment. Scott Balber, a New York lawyer who represents Kaveladze, confirmed that his client attended both the dinner in Las Vegas and the Trump Tower meeting but said he did not set up the second meeting. Trumpâ€™s son-in-law Jared Kushner, other Trump campaign aides, and Russian lawyer Natalia Veselnitskaya were also at that meeting. Lawyer Balber also said the committees were only seeking Kaveladzeâ€™s input as a witness and were not targeting him for investigation. â€œNo-one has ever told me that they have any interest in him other than as a witness,â€ Balber said. Lawyers for Trump Jr. and Kushner did not respond to requests for comment about their contacts with Kaveladze.Â A lawyer for President Trump declined to comment. One photograph from the 2013 dinner, when Trump still owned the Miss Universe pageant, shows Agalarov and his pop singer son Emin along with Trump, two Trump aides and several other people at the dining table. Another shows Kaveladze standing behind Trump and Emin Agalarov as they speak. The pictures were found by a University of California at Irvine student and blogger Scott Stedman, who posted them on Nov. 22. Aras Agalarov is a billionaire property developer in Russia who was awarded the Order of Honor by Russian President Vladimir Putin. Several U.S. officials who spoke on condition of anonymity said Muellerâ€™s team and the committees are looking for any evidence of a link between the Trump Tower meeting and the release six weeks later of emails stolen from Democratic Party organizations. They are also trying to determine whether there was any discussion at the New York meeting of lifting U.S. economic sanctions on Russia, a top priority for Putin, the officials said. Rob Goldstone, a British publicist, told Trump Jr. ahead of the New York meeting that Russian lawyer Veselnitskaya would be bringing damaging information about donations to a charity linked to Trumpâ€™s Democratic rival Hillary Clinton, according to emails later released by Trump Jr. Trump Jr. initially said the meeting was about Russian adoptions but later said it also included Veselnitskayaâ€™s promises of information on the donations to the Clinton charity. He said he ultimately never received the information, although it was later posted on the Internet. In a statement issued after meeting with the Senate Judiciary Committee on Sept. 7, Trump, Jr. said Goldstone and Veselnitskaya were in a conference room with him as well as Kaveladze and a translator. Balber said Kaveladze attended expecting to serve as a translator, although he did not do so in the end because Veselnitskaya brought her own. "]
new_sequences = [wordopt(sentence) for sentence in new_input_data]

from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer1 = Tokenizer()
tokenizer1.fit_on_texts(new_sequences)  # Assuming x_train is a list of text strings
new_sequences = tokenizer1.texts_to_sequences(new_sequences)

# Pad sequences
new_sequences = pad_sequences(new_sequences, maxlen=sequence_length)

# new_sequences = [map_tokens_to_indices(sequence, embedding_matrix, sequence_length) for sequence in new_sequences]

# Convert 'new_sequences' to a NumPy array
new_data = np.array(new_sequences)

# Pad or truncate the sequences to match the sequence length
new_data = pad_sequences(new_data, maxlen=sequence_length, padding='post', truncating='post')

# Make predictions on the new data
predictions = model1.predict(new_data)

# The 'predictions' array will contain probability scores for each class (0 and 1)
# You can convert these scores to class labels based on a threshold (e.g., 0.5)
predicted_labels = [1 if score >= 0.5 else 0 for score in predictions]

class_mapping = {0: "fake", 1: "true"}

# Use the mapping to transform the predicted labels
predicted_class_names = [class_mapping[label] for label in predicted_labels]

print(predicted_class_names)


1/1 [==============================] - 0s 419ms/step
['true']


In [1]:
pip install Flask

  Using cached flask-3.0.3-py3-none-any.whl.metadata (3.2 kB)
  Using cached itsdangerous-2.2.0-py3-none-any.whl.metadata (1.9 kB)
Using cached flask-3.0.3-py3-none-any.whl (101 kB)
Using cached itsdangerous-2.2.0-py3-none-any.whl (16 kB)
Note: you may need to restart the kernel to use updated packages.


In [59]:
from tensorflow.keras.models import load_model

# Load your trained CNN model
model1 = load_model('models/fasttext-cnn-model.h5')

In [129]:
# tokenizer = RegexpTokenizer(r'[A-Za-z]+') #to getting alpha only
# stemmer = SnowballStemmer("english")
from tensorflow.keras.preprocessing.text import Tokenizer

# Tokenize and preprocess the new input data
new_input_data = ["The fallout from Ryan Lochteâ€™s story about being robbed at gunpoint in Rio  â€”   a tale the Brazilian police said was not true  â€”   continued Monday when four companies said they would end business partnerships with Mr. Lochte, an American swimmer and   Olympic medalist. After a week of intense international media attention and anger in Brazil, the financial repercussions were swift for Mr. Lochte as Speedo USA, the luxury clothing retailer Ralph Lauren and the mattress company Airweave all announced that they would part ways with him. And Syneron Candela, a company that sells   devices, told Reuters its relationship with the swimmer ended on Sunday. Speedo USA said in a message on Twitter that it would instead donate a $50, 000 portion of Mr. Lochteâ€™s fee to a charity to help Brazilian children. â€œWhile we have enjoyed a winning relationship with Ryan for over a decade and he has been an important member of the Speedo team, we cannot condone behavior that is counter to the values this brand has long stood for,â€ the company said in its statement. On Monday, Kim Angelastro, a spokeswoman for Syneron Candela, wrote in an email, â€œWe hold our employees to high standards, and we expect the same of our business partners. â€ Mr. Lochte was a spokesman for the companyâ€™s Gentle Hair Removal brand. Through a spokeswoman, Ralph Lauren said Monday that Mr. Lochteâ€™s endorsement agreement with the clothing company had been for only the 2016 Olympics, and that his contract would not be renewed. Airweave said on Twitter that â€œafter careful consideration, we have made the decision to end our partnership with Ryan Lochte. â€ The decisions to cut ties with Mr. Lochte, 32, were the first major signs of the financial fallout for him. For the past week, he has been at the center of an international firestorm after the Brazilian police said he and three other American athletes  â€”   Jimmy Feigen, Jack Conger and Gunnar Bentz  â€”   had fabricated the account of being robbed after a   party in Rio de Janeiro. The authorities said that the swimmers had instead drunkenly vandalized a gas station bathroom, paying a security guard about $50 for the damage before leaving. Mr. Lochte originally said that the car they were traveling in had been stopped by armed men, who held a gun to his head. But his story later changed. A Brazilian judge ordered the swimmers to remain in Rio, but Mr. Lochte had already left the country. After Mr. Conger and Mr. Bentz were pulled from their flight to the United States, they told the police that the confrontation began when Lochte pulled a poster off a gas station wall. Mr. Feigen, 26, later donated $10, 800 to a charity in Rio that teaches martial arts to poor children. Mr. Lochte first issued an apology on social media  â€”   â€œI should have been much more responsible for how I handled myself,â€ he wrote  â€”   then told Matt Lauer in an interview on NBC that he had been intoxicated and that he had â€œoverexaggerated that story. â€ He has maintained that he was held at gunpoint. â€œAll we know is that there was a gun pointed in our direction, and we were demanded to give money,â€ Mr. Lochte said. Mr. Lochte, whose boyish and sometimes oafish personality had made him a commercial success in Olympics past, had headed into Rio with fewer sponsors than heâ€™d had at the London Games, according to a report by CNN Money. Mr. Lochte took home a gold medal in the   freestyle relay in Rio.", "Republicans are working overtime trying to sell their scam of a tax bill to the public as something that directly targets middle-class and working-class families with financial relief. Nothing could be further from the truth, and they re getting hammered on that repeatedly. Speaking on CNBC, Paul Ryan was going full throttle, trying to convince us that the paltry savings we re getting is actually wait for it big money.But he didn t just go with the usual talking points. With a smug look that only someone who grew up in a wealthy family can muster when talking about that which he does not know, Ryan claimed that the $2,059 more per year that families living paycheck-to-paycheck will see is extremely significant. Then he decided he had to amend that to say such savings might be nothing to a family earning $600,000 per year (true), or for people living in New York or California (false).Those are the same two states that Trump s loyal subjects insist on stripping from the 2016 vote totals to claim that Trump actually won the popular vote. Watch Ryan completely dismiss all the struggling families living in blue states below:If you re living paycheck-to-paycheck which is more than half of the people in this country and you got #2059more from a tax cut next year, that s not nothing. pic.twitter.com/8TKtrMqRa1  Paul Ryan (@SpeakerRyan) December 21, 2017Someone needs to reach through their computer or television and wipe that smugness off his face. It is the height of arrogance and insult to imply that there are no struggling families in either of those two states.Featured image via Mark Wilson/Getty Images","BREAKING : Trump Expressed Concern Over Anthony Weinerâ€™s â€œIllegal Accessâ€ to Classified Info 2 Months ago BREAKING : Trump Expressed Concern Over Anthony Weinerâ€™s â€œIllegal Accessâ€ to Classified Info 2 Months ago Breaking News By Amy Moreno October 28, 2016. Once again, Trump was right. Back in August, in a statement regarding Hillaryâ€™s carelessness handling classified documents, Trump stated that he was concerned that Weiner had â€œaccessâ€ to information he shouldnâ€™t. Now that weâ€™re learning that the FBI discovered â€œnew emailsâ€ on a â€œdeviceâ€ associated to Weiner, it looks as if Trump was right AGAIN. â€” Deplorable AJ (@asamjulian) October 28, 2016 This is a movement â€“ we are the political OUTSIDERS fighting against the FAILED GLOBAL ESTABLISHMENT! Join the resistance and help us fight to put America First! Amy Moreno is a Published Author , Pug Lover & Game of Thrones Nerd. You can follow her on Twitter here and Facebook here . Support the Trump Movement and help us fight Liberal Media Bias. Please LIKE and SHARE this story on Facebook or Twitter.  "]
new_sequences = [wordopt(sentence) for sentence in new_input_data]

from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer1 = Tokenizer()
tokenizer1.fit_on_texts(new_sequences)  # Assuming x_train is a list of text strings
new_sequences = tokenizer1.texts_to_sequences(new_sequences)

# Pad sequences
new_sequences = pad_sequences(new_sequences, maxlen=sequence_length)

# new_sequences = [map_tokens_to_indices(sequence, embedding_matrix, sequence_length) for sequence in new_sequences]

# Convert 'new_sequences' to a NumPy array
new_data = np.array(new_sequences)

# Pad or truncate the sequences to match the sequence length
new_data = pad_sequences(new_data, maxlen=sequence_length, padding='post', truncating='post')

# Make predictions on the new data
predictions = model1.predict(new_data)

# The 'predictions' array will contain probability scores for each class (0 and 1)
# You can convert these scores to class labels based on a threshold (e.g., 0.5)
predicted_labels = [1 if score >= 0.5 else 0 for score in predictions]

class_mapping = {0: "fake", 1: "true"}

# Use the mapping to transform the predicted labels
predicted_class_names = [class_mapping[label] for label in predicted_labels]

print(predicted_class_names)


1/1 [==============================] - 0s 259ms/step
['fake', 'fake', 'fake']
